In [1]:
import os

import pandas as pd
import gspread_dataframe as gs
import pyperclip as clip # copiar o texto clipboard
from time import sleep
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By

import random




# Change the directory to reflect the main repository
os.chdir("C:/Users/rsilva/Google Drive/projects/programming/automation")

#import sistemas
import sei
import functions
import page
from page import *
import helpers

% reload_ext autoreload
% autoreload 2

In [4]:
TAGS = ["@nome_interessado_maiusculas@", "@endereco_destinatario@", "@bairro_destinatario@", "@cep_destinatario@", "@cidade_destinatario@", "@validade@"]
VALUES = ["Nome", "Logradouro", ["Número", "Complemento", "Bairro"], "Cep", "Cidade", "Validade"]

In [18]:
numero = "53504.006243/2018-84"

In [19]:
gc = functions.authenticate_google("files/anatel.json")
wb = gc.open(title=numero)

In [20]:
sht = wb.get_worksheet(0)

In [21]:
df = gs.get_as_dataframe(sht, dtype=str)[:18]

In [22]:
df["Complemento"] = df["Complemento"].str.replace("nan", "")

df.drop_duplicates(inplace=True)

df.dropna(subset=["CPF"], inplace=True)

df.head()

,CPF,NOME,FISTEL,Validade,FILTRO,End_Sede,End_Correspond.,Situação,Devedor,CPF.1,...,Título de Eleitor,Situação Cadastral,Fone,Logradouro,Número,Complemento,Bairro,Cidade,Cep,Endereço_Completo
0,05234775895,ABILIO GONCALVES PRETO,02035661307,28/08/2017,=B2=K2,"RUA HUMBERTO GOZZO 120, .\n- PQ GLORIA\nCEP: 1...",RUA URUGUAI 62 VILA JUCA PEDRO \n CEP: 15800...,Atenção: Entidade devedora (Não bloqueada),Sim,052.347.758-95,...,0019319610141,REGULAR,35248858,RUA URUGUAI,62,,VILA JUCA PEDRO,CATANDUVA/SP,15800-420,RUA URUGUAI 62 VILA JUCA PEDRO \n CEP: 15800-...
1,27573375808,ALAN TEIXEIRAO HECKERT,80106161687,04/09/2017,=B3=K3,"AVENIDA INDEPENDENCIA, 75\n- CIDADE NOVA\nCEP:...",AVENIDA BRIGIDO MARCASSA 945 JARDIM PACAEMBU ...,Entidade não possui débitos,Não,275.733.758-08,...,278549690116,REGULAR,60678500,AVENIDA INDEPENDENCIA,75,,CIDADE NOVA,PINDAMONHANGABA/SP,12414-240,AVENIDA INDEPENDENCIA 75 CIDADE NOVA \n CEP: ...
2,26792767875,ALEXANDRE APARECIDO PEREIRA CANTANHEDE,80106242253,04/09/2017,=B4=K4,"RUA ROBERTO SCARPP, 428\n- JARDIM CAIEIRA\nCEP...",RUA JOAO SPANHOL 132 JD DAS ORQUIDEAS \n CEP:...,Entidade devedora (Bloqueada),Sim,267.927.678-75,...,234992960141,REGULAR,34444403,RUA ROBERTO SCARPA,284,,JD CAIEIRA,LIMEIRA/SP,13483-214,RUA ROBERTO SCARPA 284 JD CAIEIRA \n CEP: 134...
3,26052762870,ALEXANDRE UBIRAJARA ROGANTTI VERAS,80106294482,04/09/2017,=B5=K5,"RUA SANTO ANTONIO , 1822\n- VILA GALVAO \nCEP:...",RUA FRANCISCO ANTONIO MIRANDA 88 EMERLINO MAT...,Atenção: Entidade devedora (Não bloqueada),Sim,260.527.628-70,...,0177282160191,REGULAR,20860910,RUA SANTO ANTONIO,1822,,VILA GALVAO,GUARULHOS/SP,07071-000,RUA SANTO ANTONIO 1822 VILA GALVAO \n CEP: 07...
4,03636918800,ALFREDO LUIS LONGO,80106281402,28/08/2017,=B6=K6,"RUA JIM CLARK, 13\n- PARQUE DOS BANCARIOS\nCEP...",AVENIDA JAGUARE 344 JAGUARE \n CEP: 05346-00...,Entidade devedora (Bloqueada),Sim,036.369.188-00,...,0145148880124,REGULAR,22531128,RUA JIM CLARK,95,,PARQUE BANCARIO,SAO PAULO/SP,03923-165,RUA JIM CLARK 95 PARQUE BANCARIO \n CEP: 0392...


In [23]:
df.shape

(18, 26)

In [8]:
usr = "rsilva"
pwd = "Savorthemom3nts"
browser = webdriver.Firefox()

In [9]:
browser = sei.login_sei(browser, usr, pwd)

In [25]:
sei_ = sei.Sei(browser)

sei_.itera_processos()

In [26]:
p = sei_.go_to_processo(numero)

In [14]:
tipo = "RC_Oficio de Cassação"

In [15]:
def string_endereço(row):
    
    s = 'À<br>' + str(row[1]["Nome"]) + '<br>' + str(row[1]["Logradouro"]) + ", " + str(row[1]["Número"]) \
         + " " + str(row[1]["Complemento"]) + " " + str(row[1]["Bairro"]) + "<br>" +  "CEP: " \
         + str(row[1]["Cep"]) +" - " + str(row[1]["Cidade"]) + "<br><br>" + "<b>FISTEL: " \
         + str(row[1]["FISTEL"]) + "</b>"
            
    return s

In [28]:
for row in df.iloc[1:,].iterrows():    
    
    dados = {'À': string_endereço(row)}    
    
    clip.copy(dados["À"])
    
    p.incluir_oficio(tipo, dados)
        
    sleep(random.randint(60,120))

In [ ]:
for p in sei_.get_processos().values():
    
    if p["aviso"] and p["tipo"] == "Outorga: Rádio do Cidadão":
        
        p = sei_.go_to_processo(p["numero"])
        
        teste = input()
        
        if teste == 'y':
            
            p.edita_marcador()
            p.edita_postit()
            
            p.get_acoes()['conc']

In [ ]:
p = sei_.go_to_processo("53504.005936/2018-50")

In [ ]:
p.edita_marcador("RC")

In [ ]:
select = Select(p.wait_for_element(helpers.Marcador.LISTA_MARCADORES))

In [34]:
windows = sei_.driver.window_handles

sei_.driver.switch_to_window(windows[0])

sei_.driver.switch_to.default_content()